In [2]:
!pip install biom-format

In [3]:
#used for google colab

# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
from biom import load_table
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.tree import export_text
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.feature_selection import chi2,SelectKBest
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [5]:
import os
path = os.getcwd()

In [6]:
# Load the BIOM file
biom_file = path + '/Data/samples_10000_frequency_filtered.biom'
table = load_table(biom_file)

In [7]:
# print(table.ids())
# print(table.filter)
# obs = table.ids(axis='observation')
# print(obs)
# np.savetxt('obss.txt', obs, fmt='%s')
# print(table.head(10))
table

45532 x 951 <class 'biom.table.Table'> with 231299 nonzero entries (0% dense)

In [8]:
#load metadata
metadata = path + '/Data/metadata_filtered.txt'
md = pd.read_csv(metadata, sep = '\t')
md

,#SampleID,subset_diabetes,host_taxid,allergic_to_unspecified,qiita_study_id,host_common_name,non_food_allergies_sun,physical_specimen_remaining,alcohol_types_unspecified,body_site,...,allergic_to_i_have_no_food_allergies_that_i_know_of,taxon_id,subset_bmi,description,non_food_allergies_drug_eg_penicillin,subset_healthy,env_feature,body_habitat,public,sex
0,10317.000074244.69032,True,9606.0,No,10317.0,human,No,Yes,No,UBERON:feces,...,Yes,408170.0,False,American Gut Project Stool sample,No,False,human-associated habitat,UBERON:feces,Yes,female
1,10317.000097250.74745,True,9606.0,No,10317.0,human,No,Yes,No,UBERON:feces,...,Yes,408170.0,True,American Gut Project Stool sample,No,False,human-associated habitat,UBERON:feces,Yes,male
2,10317.000037974.57180,True,9606.0,TRUE,10317.0,human,FALSE,TRUE,FALSE,UBERON:feces,...,FALSE,408170.0,True,American Gut Project Stool Sample,TRUE,True,human-associated habitat,UBERON:feces,TRUE,male
3,10317.000110314.76911,False,9606.0,No,10317.0,human,No,Yes,Yes,UBERON:feces,...,Yes,408170.0,False,American Gut Project Stool sample,No,False,human-associated habitat,UBERON:feces,Yes,female
4,10317.000051141.58828,True,9606.0,FALSE,10317.0,human,FALSE,TRUE,FALSE,UBERON:feces,...,FALSE,408170.0,True,American Gut Project Stool Sample,FALSE,False,human-associated habitat,UBERON:feces,TRUE,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,10317.000054290.59891,True,9606.0,FALSE,10317.0,human,FALSE,TRUE,FALSE,UBERON:feces,...,TRUE,408170.0,True,American Gut Project Stool Sample,FALSE,True,human-associated habitat,UBERON:feces,TRUE,female
1215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
md["cardiovascular_disease"] = md["cardiovascular_disease"].map({"Diagnosed by a medical professional (doctor, physician assistant)": 1, "I do not have this condition": 0})
print(md["cardiovascular_disease"])

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
1214    0.0
1215    NaN
1216    NaN
1217    NaN
1218    NaN
Name: cardiovascular_disease, Length: 1219, dtype: float64


In [12]:
# Convert the table to a DataFrame
# data = pd.DataFrame(table.matrix_data.toarray(),
#                     index=table.ids(axis='observation'),
#                     columns=table.ids(axis='sample'))



data = pd.DataFrame(table.matrix_data.toarray().T, columns=table.ids(axis='observation'))
data['SampleID'] = table.ids(axis='sample')
data = data.sort_values(by='SampleID')
md = md.sort_values(by='#SampleID')
md = md.reset_index(drop=True)

# Drop duplicate SampleID values from data
data = data.drop_duplicates('SampleID')
md = md.drop_duplicates('#SampleID')


data['CVD'] = data['SampleID'].map(md.set_index('#SampleID')['cardiovascular_disease'])

data
# Display the updated DataFrame
final_data = data.drop('SampleID',axis =1)

In [13]:
# Split the data into features (X) and target variable (y)
X = final_data.drop('CVD', axis=1)
y = final_data['CVD']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Decision Tree Classifier

In [14]:
#Create a decision tree classifier
clf = DecisionTreeClassifier()

# Fit the classifier to the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate and print the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

# Extract specificity and sensitivity from the classification report
specificity = report.split()[-4]
sensitivity = report.split()[-2]

# Print specificity and sensitivity
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

Accuracy: 0.548951048951049
Classification Report:
              precision    recall  f1-score   support

         0.0       0.52      0.60      0.56       135
         1.0       0.58      0.50      0.54       151

    accuracy                           0.55       286
   macro avg       0.55      0.55      0.55       286
weighted avg       0.55      0.55      0.55       286

Specificity: 0.55
Sensitivity: 0.55


# Elastic Net Classifier

In [15]:
# Create an Elastic Net classifier
clf_elastic_net = ElasticNet()

# Fit the classifier to the training data
clf_elastic_net.fit(X_train, y_train)

# Make predictions on the test data
y_pred_elastic_net = clf_elastic_net.predict(X_test)

# Convert the continuous predictions to binary class labels
y_pred_binary = (y_pred_elastic_net > 0.5).astype(int)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)


# Calculate the sensitivity and specificity
true_positive = sum((y_pred_binary == 1) & (y_test == 1))
false_positive = sum((y_pred_binary == 1) & (y_test == 0))
true_negative = sum((y_pred_binary == 0) & (y_test == 0))
false_negative = sum((y_pred_binary == 0) & (y_test == 1))

sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (true_negative + false_positive)

# Generate a classification report
classification_rep = classification_report(y_test, y_pred_binary)
print(classification_rep)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.5734265734265734
              precision    recall  f1-score   support

         0.0       0.54      0.62      0.58       135
         1.0       0.61      0.53      0.57       151

    accuracy                           0.57       286
   macro avg       0.58      0.58      0.57       286
weighted avg       0.58      0.57      0.57       286

Sensitivity: 0.5298013245033113
Specificity: 0.6222222222222222


/Users/sharyu/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.513e+01, tolerance: 1.662e-02
  model = cd_fast.enet_coordinate_descent(


In [16]:
print(clf_elastic_net.coef_)
print( )
print(clf_elastic_net.intercept_)
print( )
print(clf_elastic_net.predict(X_train)[:10])
print( )
print(np.sqrt(mean_squared_error(y_test,y_pred_elastic_net)))
print( )
print(r2_score(y_test,y_pred_elastic_net))

[ 0. -0.  0. ... -0. -0.  0.]

0.4418947995595818

[0.97149367 0.47914148 0.22149031 0.11575097 0.5263735  0.95158618
 0.39725961 0.73431284 0.14282735 0.8503161 ]

0.8354148738040013

-1.8004366768831126


# Neural Networks Classifier (Multi-Layer Perceptron)

In [17]:
# Create a Neural Networks (Multi-Layer Perceptron) classifier
clf_neural_net = MLPClassifier()

# Fit the classifier to the training data
clf_neural_net.fit(X_train, y_train)

# Make predictions on the test data
y_pred_neural_net = clf_neural_net.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred_neural_net)
print("Accuracy:", accuracy)

# Calculate and print the classification report
report = classification_report(y_test, y_pred_neural_net)
print("Classification Report:")
print(report)

# Extract specificity and sensitivity from the classification report
specificity = report.split()[-4]
sensitivity = report.split()[-2]

# Print specificity and sensitivity
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

Accuracy: 0.5699300699300699
Classification Report:
              precision    recall  f1-score   support

         0.0       0.55      0.53      0.54       135
         1.0       0.59      0.61      0.60       151

    accuracy                           0.57       286
   macro avg       0.57      0.57      0.57       286
weighted avg       0.57      0.57      0.57       286

Specificity: 0.57
Sensitivity: 0.57


# Random Forest Classifier

In [18]:
# Create a Random Forest classifier
clf_random_forest = RandomForestClassifier()

# Fit the classifier to the training data
clf_random_forest.fit(X_train, y_train)

# Make predictions on the test data
y_pred_random_forest = clf_random_forest.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred_random_forest)
print("Accuracy:", accuracy)

# Calculate and print the classification report
report = classification_report(y_test, y_pred_random_forest)
print("Classification Report:")
print(report)

# Extract specificity and sensitivity from the classification report
specificity = report.split()[-4]
sensitivity = report.split()[-2]

# Print specificity and sensitivity
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

Accuracy: 0.548951048951049
Classification Report:
              precision    recall  f1-score   support

         0.0       0.52      0.76      0.61       135
         1.0       0.62      0.36      0.46       151

    accuracy                           0.55       286
   macro avg       0.57      0.56      0.54       286
weighted avg       0.57      0.55      0.53       286

Specificity: 0.57
Sensitivity: 0.53


# Support Vector Machine (SVM) Classifier

In [19]:
# Create a Support Vector Machine (SVM) classifier
clf_svm = SVC()

# Fit the classifier to the training data
clf_svm.fit(X_train, y_train)

# Make predictions on the test data
y_pred_svm = clf_svm.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred_svm)
print("Accuracy:", accuracy)

# Calculate and print the classification report
report = classification_report(y_test, y_pred_svm)
print("Classification Report:")
print(report)

# Extract specificity and sensitivity from the classification report
specificity = report.split()[-4]
sensitivity = report.split()[-2]

# Print specificity and sensitivity
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

Accuracy: 0.506993006993007
Classification Report:
              precision    recall  f1-score   support

         0.0       0.49      0.87      0.63       135
         1.0       0.61      0.18      0.28       151

    accuracy                           0.51       286
   macro avg       0.55      0.53      0.45       286
weighted avg       0.55      0.51      0.44       286

Specificity: 0.55
Sensitivity: 0.44
